# Exercise 12: Tester des modèles de classification comme RadomForest, ExtraTrees, GradientBoosting

## Part 1: Récupération des fonctions des exercices précédents

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pywt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from scipy.stats import entropy

In [2]:
def calculate_stats_features(x):
    return [np.max(x), np.min(x), np.mean(x), np.median(x), np.var(x), np.std(x), np.percentile(x, 25), np.percentile(x, 75)]

In [3]:
def calculate_zero_crossing(x):
    
    diff_signal = np.diff(np.signbit(x).astype(int))
    
    zero_crossing_indices = np.nonzero(diff_signal)[0]
    
    num_zero_crossings = len(zero_crossing_indices)
    
    return num_zero_crossings

In [4]:
def calculate_rms(x):
    squared_signal = np.square(x)

    rms = np.sqrt(np.nanmean(squared_signal))

    return rms

In [5]:
def calculate_entropy(x):
    
    unique_values, value_counts = np.unique(x, return_counts=True)
    probabilities = value_counts / len(x)

    entropy_value = entropy(probabilities, base=2)

    return entropy_value

In [6]:
def get_features(x):
    stats_features = calculate_stats_features(x)

    zero_crossing = calculate_zero_crossing(x)

    rms = calculate_rms(x)

    entropy = calculate_entropy(x)

    features = stats_features + [zero_crossing, rms, entropy]

    return features

In [8]:
def extract_features_of_signal(x, waveletname, level):
    coeff_dwt = pywt.wavedec(x, waveletname, level=level)

    all_features = []

    for coeff_set in coeff_dwt:
        features = get_features(coeff_set)

        all_features.extend(features)

    return all_features

In [9]:
df = pd.read_csv('ecg.csv')

y = df['1']

df = df.drop(columns=['0', '1'])

df_features = df.apply(lambda row: extract_features_of_signal(row, 'db4', 6), axis=1)

df_features = pd.DataFrame(df_features.tolist(), columns=[f'{i}' for i in range(len(df_features.iloc[0]))]).astype('float')

In [10]:
scaler = MinMaxScaler(feature_range=(-1, 1))

df_scaled = scaler.fit_transform(df_features)

df_train, df_test, y_train, y_test = train_test_split(df_scaled, y, test_size=0.4, random_state=9)

## Part 2: Test RandomForest

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier(random_state=42)
rf.fit(df_train, y_train)

y_train_pred = rf.predict(df_train)
y_test_pred = rf.predict(df_test)

rf_train_score = accuracy_score(y_train, y_train_pred)
rf_test_score = accuracy_score(y_test, y_test_pred)

print("Score sur l'ensemble d'entraînement (Random Forest) :", rf_train_score)
print("Score sur l'ensemble de test (Random Forest) :", rf_test_score)


Score sur l'ensemble d'entraînement (Random Forest) : 1.0
Score sur l'ensemble de test (Random Forest) : 1.0


## Part 3: Test ExtraTrees

In [12]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(random_state=9)
et.fit(df_train, y_train)

y_train_pred = et.predict(df_train)
y_test_pred = et.predict(df_test)

et_train_score = accuracy_score(y_train, y_train_pred)
et_test_score = accuracy_score(y_test, y_test_pred)

print("Score sur l'ensemble d'entraînement (Extra Trees) :", et_train_score)
print("Score sur l'ensemble de test (Extra Trees) :", et_test_score)


Score sur l'ensemble d'entraînement (Extra Trees) : 1.0
Score sur l'ensemble de test (Extra Trees) : 1.0


## Part 4: Test GradientBoosting

In [13]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

gb = GradientBoostingClassifier(random_state=9)
gb.fit(df_train, y_train)

y_train_pred = gb.predict(df_train)
y_test_pred = gb.predict(df_test)

gb_train_score = accuracy_score(y_train, y_train_pred)
gb_test_score = accuracy_score(y_test, y_test_pred)

print("Score sur l'ensemble d'entraînement (Gradient Boosting) :", gb_train_score)
print("Score sur l'ensemble de test (Gradient Boosting) :", gb_test_score)


Score sur l'ensemble d'entraînement (Gradient Boosting) : 1.0
Score sur l'ensemble de test (Gradient Boosting) : 1.0
